# Showcase of Bybit Future - Future calc
The Bybitmaster has custom methods that I coded, but can also make normal request to the Bybit API with the session attribute.

In [1]:
# Inside of the module, we use absolute imports
# Because this context is different, we need to add inside the PYTHONPATH the path of the module
import sys
sys.path.append('Bybit')

from Bybit.greekMaster import GreekMaster
import Bybit.utils


# Create a Bybit master
master = GreekMaster(demo=False, verbose=1)

2024-12-01 19:15:36,476 - greekMaster - INFO - GreekMaster initialized
2024-12-01 19:15:36,477 - greekMaster - INFO - 20
['concurrent.futures', 'concurrent', 'asyncio', 'tornado.access', 'tornado', 'tornado.application', 'tornado.general', 'stack_data.serializing', 'stack_data', 'parso', 'prompt_toolkit.buffer', 'prompt_toolkit', 'parso.python.diff', 'parso.python', 'parso.cache', 'Comm', 'ipykernel.comm', 'ipykernel', 'IPKernelApp', 'numexpr.utils', 'numexpr', 'urllib3.util.retry', 'urllib3.util', 'urllib3', 'urllib3.connection', 'urllib3.response', 'urllib3.connectionpool', 'urllib3.poolmanager', 'charset_normalizer', 'requests', 'websocket', 'pybit._websocket_stream', 'pybit', 'pybit._websocket_trading', '_plotly_utils.optional_imports', '_plotly_utils', 'pybit._http_manager', 'greekMaster.client.fetcher', 'greekMaster.client', 'greekMaster']


In [2]:
# Get all futures BTC gaps
DataWrangler = master.all_gaps_pd(inverse=False, perpetual=True, pretty=True, applyFees=True)
DataWrangler

,Buy,Sell,Gap,Coeff,APR,DaysLeft,CumVolume
0,BTCPERP,BTCUSDT,$ -90.00,0.00 %,-0.00 %,-20057,4.86B
1,BTCPERP,BTC-06DEC24,$ 200.50,0.00 %,0.00 %,5,30.98M
2,BTCPERP,BTC-13DEC24,$ 578.50,0.37 %,10.89 %,12,30.30M
3,BTCPERP,BTC-20DEC24,$ 905.00,0.71 %,13.27 %,19,30.63M
4,BTCPERP,BTC-27DEC24,$ 1260.50,1.07 %,14.79 %,26,31.78M
5,BTCPERP,BTC-31JAN25,$ 2881.50,2.74 %,16.26 %,61,30.32M
6,BTCPERP,BTC-28MAR25,$ 4984.50,4.90 %,15.22 %,117,30.69M
7,BTCPERP,BTC-27JUN25,$ 8274.50,8.28 %,14.50 %,208,30.55M
8,BTCPERP,BTC-26SEP25,$ 11442.00,11.54 %,14.06 %,299,30.49M
9,BTCUSDT,BTC-06DEC24,$ 290.50,0.08 %,5.21 %,5,4.83B


In [ ]:
master.client.fetcher.get_USDC_BTC()
# BTCUSDT	BTC-29NOV24

await master.client.fetcher.exit_both_position('BTCUSDT', 'BTC-20DEC24', 0.010, 0.014)


## Tester
Just to run some tests

In [ ]:
# 1 - Simple enter position logic (given a future and a perpetual)
# 2 - Simple exit position logic (given a future and a perpetual). Can also exit when reaching a satisfying unexpected gap
# 3 - Logic to activate both operations (given a future and a perpetual). Enter again when reaching a satisfying unexpected gap
# 4 - Logic to find the best future and perpetual to trade
# 5 - Logic to keep the loop running
# 6 - Monitoring features
# 7 - Use systemctl to launch 24/7
# We use sockets, once we selected the best local future, compared to their average gap.
# Shared state to store the latest updates

## Fetching time :)

Getting 15, 5 and 1 minute candles for everything since 01/01/2024

In [ ]:
# Future contracts
await master.save_klines(dest="store")

TypeError: GreekMaster.save_klines() missing 1 required positional argument: 'dest'

## Plot testing

Part to test the plot of the data.


In [ ]:
# Yes

## Long Spot x Short Future

Spot USDC is 0% fees and Future is 0.0550% taker fees (0% with contract delivery).
Problem is that spot cannot be used with leverage. Could borrow but that would be awkward. (tried it)

In [ ]:
Bybit.utils.load_klines_parquet("store/BTCUSDT_1.parquet", pretty=True)

Bybit.utils.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-20DEC24_1.parquet")

In [ ]:
# Long position, then short position
Bybit.utils.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-27DEC24_1.parquet", upperlimit="2024-04-08 00:00")

## Long Perpetual x Short Future

Perpetual USDT is 0.0550% taker fees and Future is 0.0550% taker fees (0% with contract delivery).
Yes we have fees, but wen can use leverage on both sides.

We use Perpetual USDT because there is more liquidity.

In [ ]:
Bybit.utils.load_klines_parquet("store/BTCUSDT_15.parquet", pretty=True)

In [ ]:
Bybit.utils.plot_compare("store/BTCUSDT_15.parquet", "store/BTC-29NOV24_15.parquet")

## Long Spot x Short Inverse Future

What if I buy spot and collateralize it to short inverse future?
We effectively capture the difference.

In [ ]:
# Keep going when get_history_pd file names are changed

## OKX 20x Leverage test

I launched a test with OKX with a short on BTCUSDT-27JUN25 and long on BTCUSDT-29NOV24.

Following section will track the the two contracts' data

In [ ]:
Bybit.utils.plot_compare("store/BTC-29NOV24_15.parquet", "store/BTC-27JUN25_15.parquet", dateLimit="2024-05-06")